# 4.8 Grouping Data & Aggregating Variables

## This script contains the following points:

### 1. Import the orders_products_merged_update dataframe 
### 2. Find the aggregated mean of the “order_number” column grouped by “department_id for this dataframe.
### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset?
### 4. Creat a loyalty flag for existing customers using the transform( ) and loc( ) functions.
### 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category and check to see if the prices of products purchased by loyal customers differ from those of regular or new customers.
### 6. Create a spending flag for each user based on the average price across all their orders using the following criteria:
### ° If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
### ° If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
### 7. Determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
### ° If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
### ° If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
### ° If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”
### 8. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

### 1. Import the orders_products_merged_update dataframe

In [2]:
path = r'/Users/berk/Instacart_Grocery_Basket_Analysis'

In [3]:
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02_Data','Prepared_Data', 'orders_products_merged_update.pkl'))

### 2. Find the aggregated mean of the “order_number” column grouped by “department_id for this dataframe.

In [4]:
df_ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset?

In [5]:
# Create a subset

df = df_ords_prods_merged[:1000000]

In [6]:
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


Not every department in the database is represented in the subset. Also, there are less observations each department compared to the entire dataset, so the mean also changes. Thus, the whole dataset's average is more important than the subset's average.

### 4. Creat a loyalty flag for existing customers using the transform( ) and loc( ) functions.

#### Using the transform( ) function

In [7]:
df_ords_prods_merged['max_order'] = df_ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [9]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10


#### Using the loc( ) function

In [10]:
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [11]:
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [12]:
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
df_ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [15]:
df_ords_prods_merged[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


### 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category and check to see if the prices of products purchased by loyal customers differ from those of regular or new customers.

In [22]:
df_ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['count', 'mean', 'min', 'max']})

prices                         
                     count       mean  min      max
loyalty_flag                                       
Loyal customer    10284093  10.386336  1.0  99999.0
New customer       6243990  13.294670  1.0  99999.0
Regular customer  15876776  12.495717  1.0  99999.0

Surprisingly, loyal customers purchase the cheapest products, with new customers paying the highest prices and regular customers following.

### 6. Create a spending flag for each user based on the average price across all their orders using the following criteria:

### ° If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

### ° If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [23]:
df_ords_prods_merged['avg_price']=df_ords_prods_merged.groupby(['user_id'])['prices'].transform(np.max)

In [24]:
df_ords_prods_merged['avg_price']

0           14.0
1           14.0
2           14.0
3           14.0
4           14.0
            ... 
32404854    24.0
32404855    24.0
32404856    14.4
32404857    14.4
32404858    24.9
Name: avg_price, Length: 32404859, dtype: float64

In [26]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,14.0
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,14.0
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,14.0


In [28]:
df_ords_prods_merged.loc[df_ords_prods_merged['avg_price'] < 10, 'spending_flag']='Low spender'

In [29]:
df_ords_prods_merged.loc[df_ords_prods_merged['avg_price'] >= 10, 'spending_flag']='High spender'

In [31]:
df_ords_prods_merged['spending_flag'].value_counts()

High spender    32372969
Low spender        31890
Name: spending_flag, dtype: int64

In [32]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,14.0,High spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender


### 7. Determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
### ° If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
### ° If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
### ° If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [33]:
df_ords_prods_merged['median_prior_orders'] = df_ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [34]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_prior_orders
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,14.0,High spender,20.5
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5


In [35]:
df_ords_prods_merged.loc[df_ords_prods_merged['median_prior_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [36]:
df_ords_prods_merged.loc[(df_ords_prods_merged['median_prior_orders'] > 10) & (df_ords_prods_merged['median_prior_orders'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [37]:
df_ords_prods_merged.loc[df_ords_prods_merged['median_prior_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [39]:
df_ords_prods_merged['order_frequency_flag'].value_counts()

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: order_frequency_flag, dtype: int64

In [38]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_prior_orders,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer


### 8. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [40]:
df_ords_prods_merged.to_pickle(os.path.join(path, '02_Data','Prepared_Data', 'orders_products_merged_updated_2.pkl'))